## Install Aplikasi

In [ ]:
import sys
from IPython.display import clear_output 
!{sys.executable} -m pip install scikit-learn==1.2.0 ipywidgets==8.0.2 pandas==1.5.0 requests
clear_output(wait=5)
print("Dependencies Installed!")

## Download Model Terbaru

In [ ]:
# Download Model
import os
import os.path
import requests
import json
from urllib.parse import urljoin

model_dir = "./model"
url = "https://oclab.github.io/aras-prediction/list.json"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

models = json.loads(requests.get(url).content)
with open("list.json", "w") as file:
    file.write(json.dumps(models, indent=2))

for key in models.keys():
    with open(models[key], "wb") as file:
        file.write(requests.get(urljoin(url, models[key])).content)
        print("Download:", key)

In [ ]:
import pickle
import pandas as pd
from ipywidgets import interact, widgets
import json
import random

# Fungsi untuk memuat model dari file .pkl
def load_model_from_pkl(file_path):
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
    return model

# Fungsi untuk melakukan prediksi pada dataset baru
def predict_new_data(model, new_data, feature_names):
    # Lakukan preprocessing pada new_data jika diperlukan
    # ...
    # Pastikan kolom sesuai dengan feature_names yang diberikan
    new_data = new_data[feature_names]
    # Lakukan prediksi menggunakan model
    predictions = model.predict(new_data)
    return predictions

# Nama file .pkl yang berisi model
with open("list.json", "r") as file:
    model_file_paths = json.loads(file.read())
# Memuat model dari file .pkl
models = {}
for model_name, model_path in model_file_paths.items():
    try:
        model = load_model_from_pkl(model_path)
        models[model_name] = model
        print(f"Model '{model_name}' berhasil dimuat.")
    except FileNotFoundError:
        print(f"File '{model_path}' tidak ditemukan.")
    except Exception as e:
        print(f"Terjadi kesalahan saat memuat model '{model_name}':", str(e))

# Fungsi untuk melakukan prediksi dengan model yang dipilih dan data yang dimasukkan oleh pengguna
def predict_with_user_input(model_name, **kwargs):
    if model_name not in models:
        print(f"Model '{model_name}' tidak ditemukan.")
        return

    # Ubah input fitur yang diberikan pengguna menjadi dataframe
    new_data = pd.DataFrame([kwargs.values()], columns=kwargs.keys())

    # Pastikan kolom sesuai dengan feature_names yang diberikan
    feature_names = [f'fitur - {i}' for i in range(1, 26)]

    # Lakukan prediksi pada data baru dengan model yang dipilih
    try:
        predictions = predict_new_data(models[model_name], new_data, feature_names)
        print("Hasil prediksi:")
        if predictions == 1:
            print ("maaf anda belum lulus")
        else:
            print ("selamat anda lulus")
    except Exception as e:
        print("Terjadi kesalahan saat melakukan prediksi:", str(e))

# Buat dropdown untuk memilih model
model_dropdown = widgets.Dropdown(
    options=list(models.keys()),
    description='Pilih Model:'
)

# Buat input fields untuk fitur-fitur menggunakan widgets.FloatText
fitur_widgets = {}
for i in range(1, 26):
    fitur_widgets[f'fitur - {i}'] = widgets.FloatText(description=f'Nilai Ke-{i}:')

# Panggil fungsi predict_with_user_input dengan model yang dipilih dan input dari pengguna
app = interact(predict_with_user_input, model_name=model_dropdown, **fitur_widgets)
